In [49]:
""" 
    正弦波使用问题二的最优函数计算误差
    非正弦波使用问题二的公式重新拟合参数，并计算误差
"""

' \n    正弦波使用问题二的最优函数计算误差\n    非正弦波使用问题二的公式重新拟合参数，并计算误差\n'

In [50]:
import pandas as pd
import numpy as np
import time

d1 = pd.read_csv("data/材料1.csv")
# d2 = pd.read_csv("data/材料2.csv")
# d3 = pd.read_csv("data/材料3.csv")
# d4 = pd.read_csv("data/材料4.csv")

d1['材料'] = 1
# d2['材料'] = 2
# d3['材料'] = 3
# d4['材料'] = 4

# data = pd.concat([d1, d2, d3, d4])
data = d1
pd.set_option('future.no_silent_downcasting', True)
data['励磁波形'] = data['励磁波形'].replace({'正弦波': 1, '三角波': 2, '梯形波': 3})

magnetic_flux_density = data.iloc[:, 4:-1].values

max_flux = np.max(magnetic_flux_density, axis=1)  # 峰值

data['max_flux'] = max_flux
lst = ['温度', '频率', "磁芯损耗", '励磁波形']
for i in range(1024):
    lst.append(i)
lst.append('材料')
lst.append('B_max')
data.columns = lst


print(data.shape)


# 筛选数据将数据分为正弦波以及其他波形
filter_wire_sin_data = data[data['励磁波形'] == 1]
filter_wire_sin_data = filter_wire_sin_data[['温度', '频率', 'B_max', '材料', '励磁波形', '磁芯损耗']]
print(filter_wire_sin_data.shape)

filter_wire_other_data = data[data['励磁波形'] != 1]
filter_wire_other_data = filter_wire_other_data[['温度', '频率', 'B_max', '材料', '励磁波形', '磁芯损耗']]
print(filter_wire_other_data.shape)
filter_wire_sin_data.head()
# filter_wire_other_data.head()

(3400, 1030)
(1067, 6)
(2333, 6)


,温度,频率,B_max,材料,励磁波形,磁芯损耗
0,25,50030,0.028849,1,1,1997.955250
1,25,50020,0.031419,1,1,2427.749830
2,25,50020,0.035535,1,1,3332.725760
3,25,50020,0.040015,1,1,4502.908007
4,25,50030,0.045028,1,1,6063.023248


In [51]:
def calc_sin(f, B_max, T):
    return 1.41 *  (f ** 1.64) * (B_max ** 2.57) * (T ** -0.56)

In [52]:
from sklearn.metrics import mean_squared_error


y_vaild = filter_wire_sin_data.iloc[:, -1]
T = filter_wire_sin_data.iloc[:, 0]
B_max = filter_wire_sin_data.iloc[:, 2]
f = filter_wire_sin_data.iloc[:, 1]

start_time = time.time()

y_pred = calc_sin(f, B_max, T)

# 计算 MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((y_vaild - y_pred) / y_vaild)) * 100
print(f"MAPE: {mape}%")

# 计算 MAE (Mean Absolute Error)
mae = np.mean(np.abs(y_vaild - y_pred))
print(f"MAE: {mae}")

# 计算 R² (决定系数)
ss_res = np.sum((y_vaild - y_pred) ** 2)
ss_tot = np.sum((y_vaild - np.mean(y_vaild)) ** 2)
r_squared = 1 - (ss_res / ss_tot)
print(f"R²: {r_squared}")

mse = mean_squared_error(y_vaild, y_pred)
print(f'MSE: {mse}')

end_time = time.time()
print("Time：", end_time - start_time)

MAPE: 14.928187751592791%
MAE: 9952.144470779149
R²: 0.9791561295333977
MSE: 618430040.5677242
Time： 0.0012919902801513672


In [53]:
"""
拟合函数: predict_loss6
最佳方法: COBYLA
最佳参数: [1.35790751 1.56139092 2.41196416 0.39924536]
最小 MAPE: 0.30764116184488594
"""
def calc_other(f, B_max, T):
    return 1.358 *  (f ** 1.561) * (B_max ** 2.412) * (T ** -0.399)

In [54]:
y_vaild = filter_wire_other_data.iloc[:, -1]
T = filter_wire_other_data.iloc[:, 0]
B_max = filter_wire_other_data.iloc[:, 2]
f = filter_wire_other_data.iloc[:, 1]

start_time = time.time()

y_pred = calc_other(f, B_max, T)

end_time = time.time()


# 计算 MAPE (Mean Absolute Percentage Error)


mape = np.mean(np.abs((y_vaild - y_pred) / y_vaild)) * 100
print(f"MAPE: {mape}%")

# 计算 MAE (Mean Absolute Error)
mae = np.mean(np.abs(y_vaild - y_pred))
print(f"MAE: {mae}")

# 计算 R² (决定系数)
ss_res = np.sum((y_vaild - y_pred) ** 2)
ss_tot = np.sum((y_vaild - np.mean(y_vaild)) ** 2)
r_squared = 1 - (ss_res / ss_tot)
print(f"R²: {r_squared}")

mse = mean_squared_error(y_vaild, y_pred)
print(f'MSE: {mse}')

print("Time Cost：", end_time - start_time)

MAPE: 30.814710863250305%
MAE: 45557.11982236878
R²: 0.9362478704782884
MSE: 9582447465.171894
Time Cost： 0.0005249977111816406


$$\LARGE P_{非正弦波} = 1.358 \cdot f^{1.561} \cdot B_{max} ^ {2.412}\cdot T^{-0.399} $$